# Tensor decomposition using Autograd

In [1]:
from sktensor import dtensor, cp_als

In [3]:
import autograd.numpy as np

In [22]:
t = np.arange(24).reshape(2, 3, 4).astype('float32')

In [23]:
t[0,0,0]=np.NAN

In [24]:
T = dtensor(t)

In [25]:
P, fit, itr, exectimes = cp_als(T, 3, init='random')

ValueError: array must not contain infs or NaNs

In [18]:
P.U[0].shape

(2, 3)

In [19]:
P.U[1].shape

(3, 3)

In [20]:
P.U[2].shape

(4, 3)

In [51]:
P

array([[-0.48671098,  1.46524661],
       [-0.51060759,  1.35143039],
       [-0.53450419,  1.23761417],
       [-0.5584008 ,  1.12379794]])

In [52]:
def kt_to_tensor(A, B, C):
    from functools import reduce
    factors = [A, B, C]
    for r in range(factors[0].shape[1]):
        vecs = np.ix_(*[u[:, r] for u in factors])
        if r:
            res += reduce(np.multiply, vecs)
        else:
            res = reduce(np.multiply, vecs)
    return res

In [53]:
%timeit combined_decomposed = kt_to_tensor(A, B, C)


10000 loops, best of 3: 45.3 µs per loop


In [54]:
%timeit new_kf_to_tensor(A, B, C)


10000 loops, best of 3: 62.4 µs per loop


In [55]:
from numba import autojit

In [56]:
au_kt_to_tensor  = autojit(new_kf_to_tensor)

In [57]:
%timeit combined_decomposed = au_kt_to_tensor(A, B, C)


The slowest run took 71534.62 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 5.01 µs per loop


In [75]:
def new_kt_to_tensor(A, B, C):
    m, n, o = A.shape[0], B.shape[0], C.shape[0]
    out = np.zeros((m, n, o))
    k_max = A.shape[1]
    for alpha in range(0, m):
        for beta in range(0, n):
            for delta in range(0, o):
                for k in range(0, k_max):
                    out[alpha, beta, delta]=out[alpha, beta, delta]+ A[alpha, k]*B[beta, k]*C[delta, k]
    return out

In [76]:
kruskal_ours = new_kt_to_tensor(A, B, C)
np.allclose(kruskal_ours, combined_decomposed)

True

In [70]:
def cost(A, B, C):
    pred = new_kt_to_tensor(A, B, C)
    gt = t
    error = (pred-gt).flatten()
    return np.sqrt(error**2).mean()

In [71]:
from autograd import multigrad

In [72]:
mg = multigrad(cost, argnums=[0, 1, 2])

In [73]:
mg(A, B, C)

AutogradHint: This error *might* be caused by assigning into arrays, which autograd doesn't support.
Sub-exception:
ValueError: setting an array element with a sequence.

In [8]:
def kt_to_tensor(A, B, C):
    factors = [A, B, C]
    for r in range(factors[0].shape[1]):
        vecs = np.ix_(*[u[:, r] for u in factors])
        if r:
            res += reduce(np.multiply, vecs)
        else:
            res = reduce(np.multiply, vecs)
    return res

In [ ]:
a = kt_to_tensor(factors)

In [ ]:
factors
new_kf_to_tensor(factors)

In [74]:
np.allclose(new_kt_to_tensor(A, B, C), kt_to_tensor(factors))

TypeError: kt_to_tensor() takes exactly 3 arguments (1 given)

In [ ]:
new_kf_to_tensor(factors).shape

In [ ]:
kt_to_tensor(factors)

In [ ]:
factors[1].shape

In [ ]:
factors[2].shape

In [ ]:
def new_kt_to_tensor(A, B, C):
    m, n, o = A.shape[0], B.shape[0], C.shape[0]
    out = np.zeros((m, n, o))
    k_max = A.shape[1]
    for alpha in range(0, m):
        for beta in range(0, n):
            for delta in range(0, o):
                for k in range(0, k_max):
                    out[alpha, beta, delta]=out[alpha, beta, delta]+ A[alpha, k]*B[beta, k]*C[delta, k]
    return out

In [90]:
np.tensordot(np.tensordot(A, B, axes=([1],[1])).T, C, axes=[1,1])

array([[-34.65443358, -30.54276333, -26.43109308, -22.31942283],
       [-42.62843541, -37.75546164, -32.88248788, -28.00951411],
       [-50.60243723, -44.96815995, -39.33388268, -33.6996054 ]])

In [78]:
A.shape

(2, 2)

In [79]:
B.shape

(3, 2)

In [83]:
C.shape

(4, 2)